In [1]:
import geopandas as gpd
import cobra.postgres.interface as pgi
import cobra.utils.filehandling as fh
import cobra.utils.jobhandling as jh
import geopandas as gpd
import cobra.utils.geodatahandling as gdh

In [11]:
download_file = 'https://download.geofabrik.de/europe/switzerland-latest.osm.pbf'
country = 'swiss'
#download_file = 'https://download.geofabrik.de/europe/great-britain-latest.osm.pbf'
#country = 'gb'
#download_file = 'https://download.geofabrik.de/europe/portugal-latest.osm.pbf'
#country = 'portugal'
#download_file = 'https://download.geofabrik.de/europe/france-latest.osm.pbf'
#country = 'france'
#download_file = 'https://download.geofabrik.de/europe/germany-latest.osm.pbf'
#country = 'Germany'

In [3]:
fileman = fh.Filemanager()
jobman = jh.Jobmanager()

In [ ]:
#jobman.delete_jobs()

In [4]:
fileman.download_plain_file(download_file, country)

In [5]:
datasets = fileman.get_data(datatype='OSM PBF')
dataset = datasets[datasets['Dataset'] == country].iloc[0]

In [6]:
dataset

Type                            OSM PBF
Dataset                           Swiss
File         switzerland-latest.osm.pbf
Path                    /download/Swiss
Size (MB)                         371.4
Name: 20, dtype: object

In [12]:
jobman.create_import_job_from_dataset(dataset, f'{country}power', style='power.style')

create_import_job_from_dataset
OSM PBF


In [19]:
jobman.get_jobs(df=True)

,UUID,Name,Job Type,Date created,Date started,Date finished,Priority,Status
0,846011c5-a9fa-438b-8bab-1cc49d0fc858,Load OSM from portugal,osm2pg,"Tue, 01 Mar 2022 14:42:39 GMT",None,"Tue, 01 Mar 2022 14:45:48 GMT",42,Finished
1,5fbc825d-507f-44b5-8f56-2cd7ef82750f,Load OSM from Swiss,osm2pg,"Tue, 01 Mar 2022 18:22:45 GMT",None,None,42,Failed
2,6277df2c-7421-4420-851b-a1acadff3d7a,Load OSM from Swiss,osm2pg,"Tue, 01 Mar 2022 18:25:10 GMT",None,"Tue, 01 Mar 2022 18:29:57 GMT",42,Finished


In [ ]:
sql = f"SELECT osm.way AS geom, osm.* FROM {country}power.planet_osm_line osm WHERE power IS NOT NULL"
pg_interface = pgi.PgInterface()
conn = pg_interface.get_connection()
powerlines = gpd.GeoDataFrame.from_postgis(sql, conn)

In [ ]:
powerlines.plot(figsize=(16,16))

In [ ]:
sql = f"SELECT osm.way AS geom, osm.* FROM {country}power.planet_osm_line osm WHERE power IS NOT NULL AND (power = 'line')"
jobman.create_new_pg_to_x(sql=sql, format='GPKG', filename=f'{country}_powerline.gpkg')

In [ ]:
jobman.get_jobs(df=True)

In [ ]:
sql = f"SELECT osm.way AS geom, osm.* FROM {country}power.planet_osm_point osm WHERE power IS NOT NULL"
i = pgi.PgInterface()
gdf_powerpoint = i.get_gdf(sql)

In [ ]:
gdf_powerpoint.plot(figsize=(16, 16))

In [ ]:
sql = f"SELECT osm.way AS geom, osm.* FROM {country}power.planet_osm_point osm WHERE power IS NOT NULL"
jobman.create_new_pg_to_x(sql=sql, format='GPKG', filename=f'{country}_powerpoint.gpkg')

In [ ]:
jobman.get_jobs(df=True)